In [121]:
import numpy as np
import random
from itertools import product
import pandas as pd
import sys
from joblib import Parallel, delayed

class Frota:
    def __init__(self, Periodo, TamanhoFrota,Demanda):
        while len(Demanda)<Periodo+1:
            Demanda.append(random.randint(1,TamanhoFrota))

        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        # self.Parametros = {}
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: 1-(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x)
        self.Estados = {}
        self.Acoes = {}
        # self.Acoes = {}
        self.Politica = []
        self.__num_estados__ = (Periodo+1)**TamanhoFrota
        self.__num_acoes__ = 2**TamanhoFrota
        self.ConstroeEstados2()
        self.ConstroeAcoes()
        
    
    def num2coefs(self,numero, base):
        coeficientes = [0]*self.TamanhoFrota
        if numero==0:
            return coeficientes
        
        cnt = 0
        while numero > 0:
            coeficiente = numero % base
            coeficientes[cnt]=coeficiente  # Insere o coeficiente à esquerda da lista
            numero //= base  # Divide o número pelo valor da base
            cnt = cnt+1
            
        return coeficientes

    def ConstroeAcoes(self):
        acoes = {}
        for i in range(self.Periodo+1):
            acoes[i]=[]
            for j in range(self.__num_acoes__):
                coefs = self.num2coefs(j,2)
                aux = self.TamanhoFrota-sum(coefs)
                if aux>=self.Demanda[i]:
                    acoes[i].append(tuple(coefs))
        
        self.Acoes = acoes 
    
    def ConstroeEstados(self):
        estados = {}
        for i in range(self.Periodo):
            for j in range(self.__num_estados__):
                coefs = self.num2coefs(j,self.Periodo+1)
                if max(coefs)<=i+1:
                    estados[(*coefs,i+1)]=0.5
        
        self.Estados = estados 
    
    
    def ConstroeEstados2(self):
        coefs = self.num2coefs(0,self.Periodo+1)
        estados = {}
        estados[(*coefs,0)] = 0.5
        estados[(*[-1]*self.TamanhoFrota,-1)] = 0
        for idx in range(self.__num_estados__):
            coefs = self.num2coefs(idx,self.Periodo+1)
            midx = max([*coefs,1])
            for j in range(midx,self.Periodo+1):
                estados[(*coefs,j)] = 0.5
        self.Estados = estados 

    def Recompensa(self,est):
        tvida = est[0]

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)
        
        # define o valor da manutenção se der defeito
        valor_defeito = -2*valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op,res_a0_de = 0.01/valor_equipamento,valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas na troca de equipamento
        res_a2 = -(1 - 0.8*valor_equipamento)
        
        res = [res_a0_op,res_a0_de,res_a1,res_a2]
        if est[0]==-1:
            res = [0,0,0,0]
        return res

    def ProximoEstado(self,estado):
        acoes = self.EstadosAcoes[estado]
        NovosEstados = {}
        for ac in acoes:
            NovoEstado = [[]]*self.TamanhoFrota
            Recompensa = [[]]*self.TamanhoFrota
            Probabilidades = [[]]*self.TamanhoFrota
            for i,a in enumerate(ac):
                rcp_aux = self.Recompensa(estado[i])
                if a==0:
                    NovoEstado[i] =[(estado[i][0]+1,estado[i][1]+1),(estado[i][0]+1,0)]
                    prob_quebrar = self.ProbQuebraFcn(estado[i][1])
                    Probabilidades[i] = [1-prob_quebrar,prob_quebrar]
                    Recompensa[i] = [rcp_aux[0],rcp_aux[1]]
                if a==1:
                    NovoEstado[i] = [(estado[i][0]+1,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[2]]
                if a==2:
                    NovoEstado[i] = [(0,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[3]]
                if a==-1:
                    NovoEstado[i] = [(-1,-1)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [0]

            # print(NovoEstado)

            probs = list(product(*Probabilidades))
            recs = list(product(*Recompensa))
            probs = [np.prod(pr) for pr in probs]
            recs = [np.sum(rc) for rc in recs]
            NovosEstados["estados"]=[(*x,estado[-1]+1) for x in list(product(*NovoEstado))]
            NovosEstados["recompensas"]=recs
            NovosEstados["probabilidades"]=probs

        return NovosEstados

class Frota2:
    def __init__(self, Periodo, TamanhoFrota,Demanda):
        while len(Demanda)<Periodo+1:
            Demanda.append(random.randint(0,TamanhoFrota))

        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.04
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn =  lambda x: 0 if x == 0 else 1 - (1 - self.ProbQuebra_0) * np.exp(-self.ProbPar * x)
        self.Estados = {}
        self.Acoes = {}
        self.Politica = []
        self.__num_estados__ = (Periodo+1)**TamanhoFrota
        self.__num_acoes__ = 2**TamanhoFrota
        self.ConstroeEstadosP()
        self.ConstroeAcoes()
        
    
    def num2coefs(self,numero, base):
        coeficientes = [0]*self.TamanhoFrota
        if numero==0:
            return coeficientes
        
        cnt = 0
        while numero > 0:
            coeficiente = numero % base
            coeficientes[cnt]=coeficiente  # Insere o coeficiente à esquerda da lista
            numero //= base  # Divide o número pelo valor da base
            cnt = cnt+1
            
        return coeficientes

    def ConstroeAcoes(self):
        acoes = {}
        acoes[-1]=[-1]
        for i in range(self.Periodo+1):
            acoes[i]=[]
            for j in range(self.__num_acoes__):
                coefs = self.num2coefs(j,2)
                aux = self.TamanhoFrota-sum(coefs)
                if aux>=self.Demanda[i]:
                    acoes[i].append(j)
        
        self.Acoes = acoes 
    
    def ConstroeEstadosP(self):
        estados = {}
        estados[(0,0)] = 0
        tam = 100000

        def worker(start, end):
            estados_locais = {}
            for idx in range(start, end):
                coefs = self.num2coefs(idx, self.Periodo+1)
                midx = max([*coefs,1])
                for j in range(midx, self.Periodo+1):
                    estados_locais[(idx,j)] = 0
            
            return estados_locais
        
        intervals = [(i, min(i + tam, self.__num_estados__)) for i in range(0, self.__num_estados__, tam)]
        results = Parallel(n_jobs=-1)(delayed(worker)(start, end) for start, end in intervals)

        for result in results:
            estados.update(result)
        
        estados[(-1,-1)] = 0
        self.Estados = estados


    def ConstroeEstados3(self):
        estados = {}
        estados[(0,0)] = -1
        estados[(-1,-1)] = 0

        for idx in range(self.__num_estados__):
            coefs = self.num2coefs(idx,self.Periodo+1)
            midx = max([*coefs,1])
            for j in range(midx,self.Periodo+1):
                estados[(idx,j)] = -1
        self.Estados = estados 

    def Recompensa(self,v,p):
        tvida = 0

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02/valor_equipamento
        
        # define o valor da manutenção se der defeito
        valor_defeito = 2*valor_manutencao
        
        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op,res_a0_de = 0,-valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao
        
        res = [res_a0_op,res_a0_de,res_a1]
        if p ==-1:
            res = [0,0,0]
        return res
    
    def coefs2num(self,coefs,base):
        num = 0
        for i,j in enumerate(coefs):
            num = num+j*base**i
        
        return num


    def ProximoEstado(self,estado):
        p = estado[-1]
        est = estado[0]
        acoes = self.Acoes[p]
        NovosEstados = {"estados":[],"recompensas":[], "probabilidades":[]}
        coefs = self.num2coefs(est,self.Periodo+1)
        if p in [self.Periodo,-1]:

            return {"estados":[[(-1,-1)]]*len(acoes),"recompensas":[[0]]*len(acoes), "probabilidades":[[1]]*len(acoes)}
        for ac in acoes:
            ac_cf = self.num2coefs(ac,2)
            NovoEstado = [[]]*self.TamanhoFrota
            Recompensa = [[]]*self.TamanhoFrota
            Probabilidades = [[]]*self.TamanhoFrota
            for i,a in enumerate(ac_cf):
                rcp_aux = self.Recompensa(coefs[i],p)
                if a == 0:
                    NovoEstado[i] =[0,coefs[i]+1]
                    prob_quebrar = self.ProbQuebraFcn(coefs[i])
                    Probabilidades[i] = [prob_quebrar,1-prob_quebrar]
                    Recompensa[i] = [rcp_aux[1],rcp_aux[0]]
                if a == 1:
                    NovoEstado[i] = [0]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[2]]
                if a == -1:
                    NovoEstado[i] = [-1]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [0]

            probs = list(product(*Probabilidades))
            recs = list(product(*Recompensa))
            probs = [np.prod(pr) for pr in probs]
            recs = [np.sum(rc) for rc in recs]
            NovosEstados["estados"].append([(self.coefs2num(x,self.Periodo+1),estado[-1]+1) for x in list(product(*NovoEstado))])
            NovosEstados["recompensas"].append(recs)
            NovosEstados["probabilidades"].append(probs)

        return NovosEstados
    
    def Proximos(self,estado,acao):
        p = estado[-1]
        est = estado[0]
        NovosEstados = {"estados":[],"recompensas":[], "probabilidades":[]}
        coefs = self.num2coefs(est,self.Periodo+1)
        if p in [self.Periodo,-1]:

            return {"estados":[(-1,-1)],"recompensas":[0], "probabilidades":[1]}
        ac_cf = self.num2coefs(acao,2)
        NovoEstado = [[]]*self.TamanhoFrota
        Recompensa = [[]]*self.TamanhoFrota
        Probabilidades = [[]]*self.TamanhoFrota
        for i,a in enumerate(ac_cf):
            rcp_aux = self.Recompensa(coefs[i],p)
            if a == 0:
                NovoEstado[i] =[0,coefs[i]+1]
                prob_quebrar = self.ProbQuebraFcn(coefs[i])
                Probabilidades[i] = [prob_quebrar,1-prob_quebrar]
                Recompensa[i] = [rcp_aux[1],rcp_aux[0]]
            if a == 1:
                NovoEstado[i] = [0]
                Probabilidades[i] = [1]
                Recompensa[i] = [rcp_aux[2]]
            if a == -1:
                NovoEstado[i] = [-1]
                Probabilidades[i] = [1]
                Recompensa[i] = [0]

        probs = list(product(*Probabilidades))
        recs = list(product(*Recompensa))
        probs = [np.prod(pr) for pr in probs]
        recs = [np.sum(rc) for rc in recs]
        NovosEstados["estados"] = [(self.coefs2num(x,self.Periodo+1),estado[-1]+1) for x in list(product(*NovoEstado))]
        NovosEstados["recompensas"] = recs
        NovosEstados["probabilidades"]=probs

        return NovosEstados

    def ValueIteration(self,tol=1.0e-10,nsim=500):
        cnt, D,norms,sim = 0, 10,[],0

        self.Politica = {est:0 for est in list(self.Estados)}
        while (D>tol and sim<nsim):
            D = 0
            V = {e:v for e,v in self.Estados.items()}
            for est,val in self.Estados.items():
                ac = self.Acoes[est[-1]]
                next = self.ProximoEstado(est)
                nst = next["estados"]
                prs = next["probabilidades"]
                rcs = next["recompensas"]
                v_aux = []
               
                for ja,a in enumerate(ac):
                    v_aux2 = 0
                    for je, e in enumerate(nst[ja]):
                        v_aux2 = v_aux2 + prs[ja][je]*(rcs[ja][je]+V[e])
                    v_aux.append(v_aux2)
                
                self.Politica[est] = ac[np.argmax(v_aux)]
                v_aux = np.max(v_aux)
                D = max([D,np.abs(v_aux-val)])
                self.Estados[est] =  v_aux

            cnt = cnt+1
            norms.append(D) 

        return norms


    def ValueIterationP(self,tol=1.0e-10,nsim=500):
        cnt, D,norms,sim = 0, 10,[],0

        self.Politica = {est:0 for est in list(self.Estados)}
        while (D>tol and sim<nsim):
            D = 0
            V = {e:v for e,v in self.Estados.items()}
            for est,val in V.items():
                ac = self.Acoes[est[-1]]
                next = self.ProximoEstado(est)
                nst = next["estados"]
                prs = next["probabilidades"]
                rcs = next["recompensas"]
                v_aux = []
               
                for ja,a in enumerate(ac):
                    v_aux2 = 0
                    for je, e in enumerate(nst[ja]):
                        v_aux2 = v_aux2 + prs[ja][je]*(rcs[ja][je]+V[e])
                    v_aux.append(v_aux2)
                
                self.Politica[est] = ac[np.argmax(v_aux)]
                v_aux = np.max(v_aux)
                D = max([D,np.abs(v_aux-val)])
                self.Estados[est] =  v_aux

            cnt = cnt+1
            norms.append(D) 

        return norms      
    
    def CaminhosProvavel(self):
        estado = (0,0)
        coefs = self.num2coefs(estado[0],self.Periodo+1)
        veiculos = [f"v_{i+1}" for i in range(self.TamanhoFrota)]
        caminho = {veiculos[i]:[t] for i,t in enumerate(coefs)}
        while estado[-1]<self.Periodo:
            aux = self.Proximos(estado,self.Politica[estado])
            ac_id = np.argmax(aux["probabilidades"])
            estado = aux["estados"][ac_id]
            coefs = self.num2coefs(estado[0],self.Periodo+1)
            for i,v in enumerate(veiculos):
                caminho[v].append(coefs[i])

        return caminho







In [122]:
ft = Frota2(12,2,[1,2,1])

In [103]:
ft.Proximos((0,0),0)

{'estados': [(0, 1), (13, 1), (1, 1), (14, 1)],
 'recompensas': [-0.08, -0.04, -0.04, 0],
 'probabilidades': [0, 0, 0, 1]}

In [125]:
ft.Acoes

{-1: [-1],
 0: [0, 1, 2],
 1: [0],
 2: [0, 1, 2],
 3: [0],
 4: [0, 1, 2],
 5: [0, 1, 2],
 6: [0, 1, 2, 3],
 7: [0],
 8: [0, 1, 2],
 9: [0],
 10: [0],
 11: [0, 1, 2],
 12: [0, 1, 2, 3]}

In [124]:
ft.CaminhosProvavel()

{'v_1': [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5],
 'v_2': [0, 1, 2, 3, 4, 5, 6, 0, 1, 2, 3, 4, 5]}

In [123]:
ft.ValueIteration()

[0.028992315450215196,
 0.019239788924995345,
 0.014496879362929885,
 0.012374738964572612,
 0.011923094363008903,
 0.011773711373169266,
 0.01174245522105475,
 0.01174245522105475,
 0.010705168109680266,
 0.010516519706241553,
 0.010495123628595601,
 0.010450774585354275,
 0]

In [90]:
seed = 6
ft.Politica[(seed+13*seed,seed)]

1

In [64]:
ft.ProximoEstado((seed+13*seed,seed))

{'estados': [[(0, 7), (91, 7), (7, 7), (98, 7)],
  [(0, 7), (91, 7)],
  [(0, 7), (7, 7)]],
 'recompensas': [[-0.08, -0.04, -0.04, 0], [-0.06, -0.02], [-0.06, -0.02]],
 'probabilidades': [[0.048946437397422896,
   0.17229198014668923,
   0.17229198014668923,
   0.6064696023091987],
  [0.2212384175441121, 0.7787615824558879],
  [0.2212384175441121, 0.7787615824558879]]}

In [67]:
ft.num2coefs(91,13)

[0, 7]

In [71]:
ft.Politica[(91,7)]

2

In [ ]:
ft.ProximoEstado((0,0))

In [ ]:
len(ft.ProximoEstado((194,11))["estados"])

In [ ]:
ft.Acoes

In [ ]:
x = [v[0] for v in res]
y =  [v[1] for v in res]

In [ ]:
x